# Implementation of an bully algorithm without the Django framework
### This is a test, to know how it's build without the extra layers -> object based Python classes

In [20]:
class truck(object):
    def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broke: bool):
        self.truckId = truckId
        self.convoyPosition = convoyPosition
        self.convoyLeaderId = convoyLeaderId
        self.speed = speed
        self.broke = broke
    
    def print_alive(self):
        print(self.broke)
    
    def __get_truckId__(self):
        return self.truckId
    
    def __set_convoyPosition__(self, convoyPosition):
        self.convoyPosition = convoyPosition
        
    def __get_convoyPosition__(self):
        return self.convoyPosition
    
    def __set_convoyLeaderId__(self, convoyLeaderId):
        self.convoyLeaderId = convoyLeaderId
        
    def __get_convoyLeaderId__(self):
        return self.convoyLeaderId
    
    def __set_speed__(self, speed):
        self.speed = speed
        
    def __get_speed__(self):
        return self.speed
    
    def __toggle_broke__(self):
        self.broke = not self.broke
    
    def __get_broke__(self):
        return self.broke
    
    def new_convoy_positioning(self):
        pass

In [35]:
def createTrucks(amount: int):
    convoyLeaderId = 0
    speed = 35.0
    broke = False
    trucks = {}
    
    for truckId_convoyPosition in range(amount):
        sel_truck = truck(truckId_convoyPosition, truckId_convoyPosition, convoyLeaderId, speed, broke)
        trucks[truckId_convoyPosition] = sel_truck
    return trucks

In [38]:
def de_activate(trucks, truckIdToDeactivate):
    trucks[truckIdToDeactivate].__toggle_broke__()

In [37]:
t = createTrucks(5)
ts = bully(t, 3)

0
1
2
3
4


In [36]:
def bully(trucks, pollRequestId: int):
    for key in trucks:
        print(key)
# rekursives arbeiten mit dem poll, d.h. die poll wird von unten nach oben angestossen und dann rekursiv ueber
# die element ids ausgefuehrt, nachdem die rueckmeldung gegeben wurde